In [ ]:
import numpy as np
import skimage.transform as trans
import skimage.io as io

import numpy as np 
import os
import glob

import tensorflow as tf
seed_value= 666
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

## Folder List

In [ ]:
#cfg file content

#whole brain
non_T1C_data = "../input/chimei-tumor-segmentation/PSPF_voc_data/non_PR/T1c/JPEGImages"  
PR_T1C_data = "../input/chimei-tumor-segmentation/PSPF_voc_data/PR/T1c/JPEGImages" 

non_T1_data = "../input/chimei-tumor-segmentation/PSPF_voc_data/non_PR/T1/JPEGImages" 
PR_T1_data = "../input/chimei-tumor-segmentation/PSPF_voc_data/PR/T1/JPEGImages" 

non_Flair_data = "../input/chimei-tumor-segmentation/PSPF_voc_data/non_PR/Flair/JPEGImages" 
PR_Flair_data = "../input/chimei-tumor-segmentation/PSPF_voc_data/PR/Flair/JPEGImages" 

non_T2_data = "../input/chimei-tumor-segmentation/PSPF_voc_data/non_PR/T2/JPEGImages" 
PR_T2_data = "../input/chimei-tumor-segmentation/PSPF_voc_data/PR/T2/JPEGImages" 
#mask (ground truth)
non_T1C_target = "../input/chimei-tumor-segmentation/PSPF_voc_data/non_PR/T1c/SegmentationClassPNG"
PR_T1C_target = "../input/chimei-tumor-segmentation/PSPF_voc_data/PR/T1c/SegmentationClassPNG"

non_T1_target = "../input/chimei-tumor-segmentation/PSPF_voc_data/non_PR/T1/SegmentationClassPNG"
PR_T1_target = "../input/chimei-tumor-segmentation/PSPF_voc_data/PR/T1/SegmentationClassPNG"

non_Flair_target = "../input/chimei-tumor-segmentation/PSPF_voc_data/non_PR/Flair/SegmentationClassPNG" 
PR_Flair_target = "../input/chimei-tumor-segmentation/PSPF_voc_data/PR/Flair/SegmentationClassPNG"

non_T2_target = "../input/chimei-tumor-segmentation/PSPF_voc_data/non_PR/T2/SegmentationClassPNG" 
PR_T2_target = "../input/chimei-tumor-segmentation/PSPF_voc_data/PR/T2/SegmentationClassPNG"

In [ ]:
"""load data"""
#patients in data_folcder is cinsistent with traget_folder
non_name_set = set()
PR_name_set = set()
name_set = set()
total_data_path = [non_T1C_data , non_T1_data , non_Flair_data , non_T2_data , PR_T1C_data , PR_T1_data , PR_Flair_data  , PR_T2_data]
count1 = 0
for folder_name in total_data_path :
    for name in os.listdir(folder_name) :
        name = name.strip('.jpg').strip('.json')
        if name == 'ipynb_checkpoint' :
            continue
        name_set.add(name)
        if count1 <= 3 :
            non_name_set.add(name)
        else :
            PR_name_set.add(name)
    count1 += 1
#print(name_set)

In [ ]:
total_data_list = []
total_target_list =[] 
for non_name , pr_name in zip(non_name_set , PR_name_set) :
    non_name1 = non_name + ".jpg"
    non_name2 = non_name + ".png"
    pr_name1 = pr_name + ".jpg"
    pr_name2 = pr_name + ".png"
    total_data_list.append(os.path.join(non_T1C_data , non_name1))
    total_data_list.append(os.path.join(non_T1_data , non_name1))
    total_data_list.append(os.path.join(non_Flair_data , non_name1))
    total_data_list.append(os.path.join(non_T2_data , non_name1))
    total_target_list.append(os.path.join(non_T1C_target , non_name2))
    total_target_list.append(os.path.join(non_T1_target , non_name2))
    total_target_list.append(os.path.join(non_Flair_target , non_name2))
    total_target_list.append(os.path.join(non_T2_target , non_name2))
    
    total_data_list.append(os.path.join(PR_T1C_data , pr_name1))
    total_data_list.append(os.path.join(PR_T1_data , pr_name1))
    total_data_list.append(os.path.join(PR_Flair_data , pr_name1))
    total_data_list.append(os.path.join(PR_T2_data , pr_name1))
    total_target_list.append(os.path.join(PR_T1C_target , pr_name2))
    total_target_list.append(os.path.join(PR_T1_target , pr_name2))
    total_target_list.append(os.path.join(PR_Flair_target , pr_name2))
    total_target_list.append(os.path.join(PR_T2_target , pr_name2)) 

In [ ]:
train_data_list = total_data_list[0 : int(len(total_data_list)*0.9)] #90%data
train_target_list =  total_target_list[0 : int(len(total_target_list)*0.9) ] #90%target

test_data_list = total_data_list[int(len(total_data_list)*0.9) : len(total_data_list)] #90%data
test_target_list = total_target_list[int(len(total_target_list)*0.9) : len(total_target_list)] #90%target

val_img = train_data_list[0 : int(0.2* len(train_data_list))]
val_lbl = train_target_list[0 : int(0.2* len(train_target_list))]
train_data_list = train_data_list[int(0.2* len(train_data_list)) : ]
train_target_list = train_target_list[int(0.2* len(train_target_list)) : ]
print(len(train_data_list))
print(len(train_target_list))
print(len(test_data_list))
print(len(test_target_list))
print(len(val_img))
print(len(val_lbl))

## Data Preprocess

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader , Dataset 

import matplotlib.pyplot as plt
import cv2
import torchvision.transforms as T 
from torchvision.transforms.functional import hflip , vflip
import torch

class Total_dataset(Dataset):
    def __init__(self, inputs: list, targets: list, transform=None) -> None:
        super().__init__()
        self.inputs = inputs
        self.targets = targets
        self.transform = transform
    
    def __len__(self):
        return len(self.inputs)
    def __getitem__(self , index : int  , transform = None):
        #是否有辦法用path + file name取用data
        input_img = self.inputs[index]
        target_img = self.targets[index]
        
        x_data , y_data = cv2.imread(input_img) , cv2.imread(target_img)
        
        #print(x_data.shape)
        #print(y_data.shape)
        
        
        #resize
        if x_data.shape != (512 , 512 , 3) :
            x_data = cv2.resize(x_data , (512 , 512))
        if y_data.shape != (512 , 512 , 3) :
            y_data = cv2.resize(y_data , (512 ,512))

        # turn to torch (typecasting)

        #data augmentaion if true
        if self.transform != None : 
            #x_data , y_data = self.transform(x_data , y_data)
            for t in self.transform :
                if t == "T.Normalize([1 , 1 , 1] , [1 , 1 , 1])" :
                    """
                    x y mean0 tensor(0.2302) tensor(0.0111)
                    x y mean1 tensor(0.1842) tensor(0.0111)
                    x y mean2 tensor(0.1870) tensor(0.0111)
                    x y std0 tensor(0.2334) tensor(0.0737)
                    x y std01 tensor(0.1999) tensor(0.0737)
                    x y std2 tensor(0.1987) tensor(0.0737)
                    """
                    #normalized with whole dataset mean and std (not batchwise)
                    img_normalize = T.Normalize([0.2302 , 0.1842 , 0.1870] , [0.2334 , 0.1999 , 0.1987])
                    x_data = img_normalize(x_data)
                    label_normalize = T.Normalize([0.0111 , 0.0111 , 0.0111 ] , [0.0737 , 0.0737 , 0.0737]) 
                    y_data = label_normalize(y_data)
                elif t == "T.RandomHorizontalFlip(p=1)" :
                    if torch.rand(1)< 0.5 :
                        x_data = hflip(x_data)
                        y_data = hflip(y_data)
                elif t == "T.RandomVerticalFlip(p=1)" :
                    if torch.rand(1)< 0.5 :
                        x_data = vflip(x_data)
                        y_data = vflip(y_data)
                        

                elif t == "T.RandomRotation(degrees=(360 , 360))" : 
                    if torch.rand(1)< 0.5 :
                        angle = int(torch.rand(1)*360)
                        rotate_funct = T.RandomRotation(degrees=(angle, angle))
                        x_data = rotate_funct(x_data)
                        y_data = rotate_funct(y_data)
                else :
                    x_data = t(x_data)
                    y_data =t(y_data)
                
        #x_data, y_data = torch.from_numpy(x_data).type(torch.float32), torch.from_numpy(y_data).type(torch.float32)
        #x_data = torch.permute(x_data , (1, 2 , 0))
        #y_data = torch.permute(y_data , (1 , 2 , 0))
        y_data = y_data[2, : , :] #edit to the last channel
        y_data = np.expand_dims( y_data, axis= 0) #(512 512) -> (512 512 1)
        return x_data , y_data  #torch

In [ ]:
"""mean std calculated"""
#total dataset
total_dataset = Total_dataset(total_data_list ,
                              total_target_list,
                              transform= [T.ToTensor()]
)
print(len(total_dataset))
total_dataloader = DataLoader(total_dataset , batch_size= len(total_dataset) , shuffle= False)
#total_dataloader = DataLoader(total_dataset , batch_size= len(total_data_list) , shuffle= False)

x , y = next(iter(total_dataloader))
print("x_shape", x.shape)
print("y_shape" , y.shape)

print("x y mean0" ,x[0].mean() , y[0].mean()) #ch1
print("x y mean1" ,x[1].mean() , y[1].mean()) #ch2
print("x y mean2" ,x[2].mean() , y[2].mean()) #ch3
print("x y std0" , x[0].std() , y[0].std()) #ch1
print("x y std01" , x[1].std() , y[1].std()) #ch2
print("x y std2" , x[2].std() , y[2].std() ) #ch3 

#img visualize after to_tensor
"""for index in range(5) :
    #cv2.imshow(f"{index}.jpg" , x[index , : , : , :].detach().cpu().numpy())
    #cv2.imshow(f"{index}.png" , y[index , : , : , : ].detach().cpu().numpy())
    plt.imshow(x[index , : , : , :])
    plt.show()
    plt.imshow(y[index , : , : , :])
    plt.show()
"""

In [ ]:
"""data aug"""
normal = "T.Normalize([1 , 1 , 1] , [1 , 1 , 1])"
v_flip = "T.RandomVerticalFlip(p=1)"
h_flip = "T.RandomHorizontalFlip(p=1)"
r_rotate = "T.RandomRotation(degrees=(360 , 360))"
#note : rewrite every probability functions in Total_dataset()
train_dataset = Total_dataset(train_data_list,
                             train_target_list ,
                             transform= [T.ToTensor() ,
                             normal , 
                             v_flip , 
                             h_flip , 
                             r_rotate,
                             #T.RandomAutocontrast(p=1) ,
                             ])
                                             
training_dataloader = DataLoader(train_dataset,
                                 batch_size=8,
                                 shuffle=True)


val_dataset = Total_dataset(val_img ,
                            val_lbl ,
                            transform= [ T.ToTensor() , normal , 
                                       # T.RandomAutocontrast(p=1)
                                       ])

val_dataloader = DataLoader(val_dataset,
                                 batch_size=4,
                                 shuffle=True)

#, T.RandomAutocontrast(p=1)
test_dataset = Total_dataset(test_data_list ,
                            test_target_list ,
                            transform= [ T.ToTensor() , normal])

testing_dataloader = DataLoader(test_dataset,
                                 batch_size=1,
                                 shuffle=True)


#train_x, train_y = next(iter(training_dataloader))

## Model

In [ ]:
import torch.nn as nn
import torch.functional as F
import torch


class UNET2D(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.CONV1 = nn.Conv2d(3 , 64 , 3 , padding=(1, 1))#conv9
        self.CONV1_2 = nn.Conv2d(64 , 64 , 3 , padding=(1, 1))#conv9
        self.CONV2 = nn.Conv2d(64 , 128 , 3 , padding=(1, 1))#conv8
        self.CONV2_2 = nn.Conv2d(128 , 128 , 3 , padding=(1, 1))#conv8
        self.CONV3 = nn.Conv2d(128 , 256 , 3 , padding=(1, 1))#conv7
        self.CONV3_2 = nn.Conv2d(256 , 256 , 3 , padding=(1, 1))#conv7
        self.CONV4 = nn.Conv2d(256 , 512 , 3  , padding=(1, 1))#conv6
        self.CONV4_2 = nn.Conv2d(512 , 512 , 3  , padding=(1, 1))#conv6
        self.CONV5 = nn.Conv2d(512 , 1024 , 3 , padding=(1, 1))
        self.CONV5_2 = nn.Conv2d(1024 , 1024 , 3 , padding=(1, 1))
        self.CONV9_1 = nn.Conv2d(64 , 2 , 3 , padding=(1, 1))
        self.CONV9_2 = nn.Conv2d(2 , 1 , 1) #use sigmoid as activation
        self.POOL = nn.MaxPool2d(2, 2)
        self.DROP = nn.Dropout2d(p = 0.5)
        self.UP = nn.Upsample(scale_factor=2, mode='nearest')
        self.CONV5R = nn.Conv2d(1024 , 512 , 2 , padding="same")
        self.CONV5R_1 = nn.Conv2d(1024 , 512 , 3 , padding="same")
        self.CONV5R_2 = nn.Conv2d(512 , 512 , 3 , padding="same")
        self.CONV2R = nn.Conv2d(512 , 256 , 2 , padding="same")
        self.CONV2R_1 = nn.Conv2d(512 , 256 , 3 , padding="same")
        self.CONV2R_2 = nn.Conv2d(256 , 256 , 3 , padding="same")
        self.CONV3R = nn.Conv2d(256 , 128 , 2 , padding="same")
        self.CONV3R_1 = nn.Conv2d(256 , 128 , 3 , padding="same")
        self.CONV3R_2 = nn.Conv2d(128 , 128 , 3 , padding="same")
        self.CONV_last1 = nn.Conv2d(128 , 64 , 2 , padding="same")
        self.CONV_last2 = nn.Conv2d(128 , 64 , 3 , padding="same")
        self.CONV_last3 = nn.Conv2d(64 , 64 , 3 , padding="same")
        self.CONV_last4 = nn.Conv2d(64 , 2 , 3 , padding="same")
        self.CONV_last5 = nn.Conv2d(2 , 1 , 1 , padding="same")
        self.SIGM = nn.Sigmoid()
    

    def forward(self, input):
        
        """conv1 = nn.ReLU(self.CONV1(input))
        conv1 = nn.ReLU(self.CONV1(conv1))"""
        conv1 = nn.ReLU()(self.CONV1(input))
        conv1 = nn.ReLU()(self.CONV1_2 (conv1))
        pool1 = self.POOL(conv1)
        #print("pool1" , pool1.shape)


        conv2 = nn.ReLU()(self.CONV2(pool1))
        conv2 = nn.ReLU()(self.CONV2_2(conv2))
        pool2 = self.POOL(conv2) 
        #print("pool2" , pool2.shape)


        conv3 = nn.ReLU()(self.CONV3(pool2))
        conv3 = nn.ReLU()(self.CONV3_2(conv3))
        pool3 = self.POOL(conv3)
        #print("pool3" , pool3.shape)

        
        conv4 = nn.ReLU()(self.CONV4(pool3))
        conv4 = nn.ReLU()(self.CONV4_2(conv4))
        drop4 = self.DROP(conv4)
        pool4 = self.POOL(drop4)
        #print("pool4" , pool4.shape)


        conv5 = nn.ReLU()(self.CONV5(pool4))
        conv5 = nn.ReLU()(self.CONV5_2(conv5))
        drop5 = self.DROP(conv5)
        #print("drop5" , drop5.shape)
        #############################################
        up6 = self.CONV5R(self.UP(drop5))
        merge6 = torch.cat((drop4 , up6) , dim = 1)
        #print("merge6" , merge6.shape)
        conv6 = self.CONV5R_1(merge6)
        conv6 = self.CONV5R_2(conv6)
        #print("conv6" , conv6.shape)

        up7 = self.CONV2R(self.UP(conv6))
        merge7 = torch.cat((conv3 , up7) , dim = 1)
        conv7 = self.CONV2R_1(merge7)
        conv7 = self.CONV2R_2(conv7)

        up8 = self.CONV3R(self.UP(conv7))
        merge8 = torch.cat((conv2 , up8) , dim = 1)
        conv8 = self.CONV3R_1(merge8)
        conv8 = self.CONV3R_2(conv8)

        up9 = self.CONV_last1(self.UP(conv8))
        merge9 = torch.cat((conv1 , up9) , dim = 1)
        #merge9 (1, 512, 512, 128)
        conv9 = self.CONV_last2(merge9)
        conv9 = self.CONV_last3(conv9)
        conv9 = self.CONV_last4(conv9)
        temp9 = self.CONV_last5(conv9)
        conv10 = self.SIGM(temp9)
        
        return conv10


unet = UNET2D()
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device1)
#in-place
unet.to(device1)
#unet.cuda()


In [ ]:
"""test"""
#input torch.Size([32, 3 , 512 , 512])
"""
input = torch.randn(1 , 3 , 512 , 512)
unet.forward_test(input)
"""

# (Training) Loss function and optimizer

In [ ]:
learning_rate = 1e-4


In [ ]:
#### taining two ############
import torch.optim as optim
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(unet.parameters(), lr = 0.01, momentum=0.99)
for epoch in range(30):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(training_dataloader, 0) :
        # Resizing the outputs and label to caculate pixel wise softmax loss
        optimizer.zero_grad() 
        inputs, labels = data #batched
        inputs = inputs.to(device1)
        labels = labels.to(device1)
        
        outputs = unet(inputs)
        
        outputs = outputs.to(device1)
        print(outputs.shape)
        break
        """loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0"""

print('Finished Training')

In [ ]:
#### taining one ############
import torch.optim as optim
optimizer = optim.Adam(unet.parameters() , lr = learning_rate)
loss_function = nn.BCELoss()

for epoch in range(3):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(training_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data #batched
        inputs = inputs.to(device1)
        labels = labels.to(device1)
        #print(inputs.shape , labels.shape)
        #print(inputs.shape)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = unet(inputs)
        outputs = outputs.to(device1)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0

print('Finished Training')

In [ ]:
"""clean ram"""
import os
import gc
import psutil
import pandas as pd
import numpy  as np 


process = psutil.Process(os.getpid())
print('start', process.memory_info().rss)

mtx  = np.zeros((100000000, 1)).astype('int')
df   = pd.DataFrame(mtx)

print('df created', process.memory_info().rss)

del df;  gc.collect()

print('df deleted', process.memory_info().rss)

del mtx;  gc.collect()

print('mtx deleted', process.memory_info().rss)

In [ ]:
torch.save(unet , "/kaggle/working/unet_model1-2_2")

In [ ]:
##view training file sets ###
## 確認一下training_loader 有沒有沒配對好 ##
## 確認一下到底要不要再加上T.RandomAutocontrast(p=1) ##
## lossfunction 確認一下 方式##
## accuracy 跟 callback function##
## whole dataset normalization 也是怪怪的 ，有沒有好方法可以不要一定餵定值##
%matplotlib inline
for data in training_dataset :
    inputs, labels = data #batched
    

In [ ]:
"""import torch
torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")"""
print(device1)
%matplotlib inline
model = torch.load("./unet_model1-2_2")
model.to(device1)
for data in testing_dataloader:
    
    inputs, labels = data #batched
    inputs = inputs.to(device1)
    labels = labels.to(device1)
    test_output = model(inputs)
    #print(test_output.cpu().data.numpy())
    test_output = np.squeeze(test_output.cpu().data.numpy() , axis = 0)
    test_output = np.transpose(test_output , (2 , 1 , 0))
    labels = np.squeeze(labels.cpu().data.numpy() , axis = 0)
    labels = np.transpose(labels , (2 , 1 , 0))
    plt.imshow(test_output)
    plt.title("output")
    plt.show()
    plt.imshow(labels)
    plt.title("label")
    plt.show()
    
